In [1]:
import os
import sys
sys.path.append('../../')

import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
import swifter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from datetime import datetime, timedelta
from tqdm import tqdm, tqdm_notebook
from category_encoders import TargetEncoder
import core.config as conf

In [2]:
tqdm_notebook().pandas()

/home/nyongja/anaconda3/envs/nh2021/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


0it [00:00, ?it/s]

## 01. Train Data Load

In [3]:
## load preprocessing data (past_d)
train_df = pd.read_csv('./result/train_past_d.csv')
test_df = pd.read_csv('./result/test_past_d.csv')

In [4]:
train_df

,act_id,iem_cd,byn_dt,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,past_d,hist_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,1,9,3,2,2,9,5,5,8,101,1,1,1,195.0,6.6
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,1,9,3,2,2,9,5,5,8,1361,9,1,1,86.0,48.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,1,9,3,2,2,9,5,5,8,2530,12,2,99,6.0,3.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22,1,9,3,2,2,9,5,5,8,1969,8,2,1,22.0,13.2
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324,1,9,3,2,2,9,5,5,8,1696,10,3,99,347.0,194.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A260660,20180831,1,1,4,4,2,2,3,4,3,8,1752,10,3,99,1.0,0.6
681468,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A271980,20201027,1,1,4,4,2,2,3,4,3,8,2344,8,2,99,1.0,0.6
681469,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A289080,20181121,1,1,4,4,2,2,3,4,3,8,521,2,2,99,1.0,0.6
681470,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A307930,20200214,1,1,4,4,2,2,3,4,3,8,2460,2,3,99,1.0,0.6


In [5]:
path = conf.data_path

In [5]:
train_df = pd.read_csv(f'{path}'+'stk_hld_train.csv')

In [6]:
train_df.head()

,act_id,iem_cd,byn_dt,hold_d
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324


In [7]:
cus_df = pd.read_csv(f'{path}'+'cus_info.csv')

In [8]:
cus_df.head()

,act_id,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd
0,64aae8dd71e5c0761000db9f9a6779e504e7a4aa9dc097...,1,4,99,1,2,3,3,6,16
1,5f7c3a8f37d9c268d06130ff0be5d32a1b9ef68c13049f...,1,6,4,4,2,5,2,6,1
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2,7,4,3,2,5,5,6,9
3,6d497facfa1ea5901b827335553331f8555fec02a8184f...,2,6,4,4,8,5,3,6,16
4,b727c78d2cfc246c97b677f29a034399a0c7e7873fff44...,1,5,2,2,2,5,5,5,16


In [9]:
iem_df = pd.read_csv(f'{path}'+'iem_info_20210902.csv')

In [10]:
iem_df.head()

,iem_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,A000020,동화약품,8,2,99
1,A000080,하이트진로,14,2,1
2,A000180,성창기업지주,5,3,99
3,A000227,유유제약2우B,8,99,99
4,A000325,노루홀딩스우,2,99,99


### 종목 Label Encoding

In [11]:
L_encoder = LabelEncoder()
L_encoder.fit(iem_df["iem_krl_nm"])

LabelEncoder()

In [12]:
iem_df['iem_krl_nm'] = L_encoder.transform(iem_df["iem_krl_nm"])

In [13]:
train_df = pd.merge(left = train_df, right = cus_df, how='left', on='act_id')

In [14]:
train_df = pd.merge(train_df, iem_df, how='left', on='iem_cd')

## 02. Test Data Load

In [15]:
test_df = pd.read_csv(f'{path}'+'stk_hld_test.csv')

In [16]:
test_df = pd.merge(left = test_df, right = cus_df, how='left', on='act_id')

In [17]:
test_df = pd.merge(test_df, iem_df, how='left', on='iem_cd')

In [18]:
test_df.head()

,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8,418,4,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8,2230,10,3,99
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8,1515,13,2,99
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8,2681,13,3,99
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8,2450,13,1,2


## 03. Stock Hist

In [19]:
stk_df = pd.read_csv(f'{path}'+'stk_bnc_hist.csv')

In [20]:
stk_df.head()

,act_id,bse_dt,iem_cd,bnc_qty,tot_aet_amt,stk_par_pr
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200820,A008770,40.0,2828000.0,5000.0
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A008770,20.0,1390000.0,5000.0
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20160104,A005940,311.0,2982490.0,5000.0
3,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200814,A005930,40.0,2320000.0,100.0
4,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A005930,20.0,1028000.0,100.0


## 04. Data Preprocessing

### hist d : 

In [6]:
train_df["hist_d"] = train_df["hold_d"]*0.6

In [6]:
def working_day(s_dt, e_dt) : # 공휴일 제외
    s_dt = str(s_dt).split('.')[0]
    e_dt = str(e_dt).split('.')[0]
    s_dt = s_dt[:4] + '-' + s_dt[4:6] + '-' + s_dt[6:]
    e_dt = e_dt[:4] + '-' + e_dt[4:6] + '-' + e_dt[6:]
    return np.busday_count(s_dt, e_dt, weekmask='1111100')

### External Data Featuring
- 종목 별 volume feature 추가

In [7]:
external_path = conf.external_path

In [8]:
stk_mean_df = pd.read_csv(external_path+'mean_stockdata.csv')

In [9]:
stk_mean_df['code'] = stk_mean_df.apply(lambda x : 'A' + x['code'], axis = 1)

In [10]:
stk_mean_df

,code,oepn,high,low,close,volume,chage
0,A000020,10628.321108,10875.158924,10395.338223,10611.340668,4.514417e+05,0.001178
1,A000080,13718.842105,13873.710526,13566.907895,14156.565789,1.592198e+06,0.000800
2,A000180,3205.013040,3268.528117,3137.779136,3229.211084,1.243295e+06,-0.000945
3,A000227,13186.233089,13408.596577,12970.257539,13402.982885,3.864996e+04,-0.000098
4,A000325,18285.819071,18510.423798,18045.867971,18283.928280,1.757751e+05,0.000082
...,...,...,...,...,...,...,...
2807,A156080,10585.937500,10747.812500,10474.375000,10595.937500,2.414732e+05,0.007861
2808,A160980,10113.500000,10179.500000,9927.500000,10037.000000,1.963900e+04,0.000063
2809,A161580,10127.000000,10232.000000,9951.000000,10080.000000,8.949426e+05,0.001339
2810,A163730,10156.500000,10307.250000,10123.000000,10264.500000,9.705650e+04,0.016875


In [11]:
train_df = pd.merge(left = train_df, right = stk_mean_df[['code', 'volume']], how='left', left_on='iem_cd', right_on = 'code')

In [12]:
test_df = pd.merge(left = test_df, right = stk_mean_df[['code', 'volume']], how='left', left_on='iem_cd', right_on = 'code')

### Target Encoding
종목 별 hold_d를 사용해서 종목 코드를 target encoding

In [13]:
target_encoder = TargetEncoder()

In [14]:
target_encoder.fit(train_df['iem_cd'], train_df['hold_d'])

TargetEncoder(cols=['iem_cd'])

In [15]:
train_df['iem_cd_te'] = target_encoder.transform(train_df['iem_cd'], train_df['hold_d'])

In [16]:
test_df['iem_cd_te'] = target_encoder.transform(test_df['iem_cd'], test_df['hold_d'])

In [17]:
test_df = test_df.fillna(0)

In [18]:
train_df = train_df.fillna(0)

In [20]:
#target_encoder.transform(train_df['ivs_icn_cd'], train_df['hold_d'])

### past_d 계산
user 별 종목 past_d

In [36]:
id_stk_dur_df = pd.read_csv(conf.data_path + 'id_stk_dur_list.csv')

In [37]:
train_df = pd.merge(train_df, id_stk_dur_df[['act_id', 'iem_cd', 'act_past_d_mean']], how = 'left', on = ['act_id', 'iem_cd'])

In [38]:
test_df = pd.merge(test_df, id_stk_dur_df[['act_id', 'iem_cd', 'act_past_d_mean']], how = 'left', on = ['act_id', 'iem_cd'])

### 투자성향 별 past_d 계산

In [19]:
ivs_past_df = pd.read_csv(conf.data_path + 'preprocessing/ivs_past_d_mean.csv')

In [20]:
train_df = pd.merge(train_df, ivs_past_df, how = 'left', on = ['iem_cd', 'ivs_icn_cd'])

In [21]:
test_df = pd.merge(test_df, ivs_past_df, how = 'left', on = ['iem_cd', 'ivs_icn_cd'])

In [22]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

## hist_d

In [23]:
def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

In [24]:
def hist_d_half(buy, hold_d) :
    buy_d = str(buy)
    buy_d = datetime(int(buy_d[:4]), int(buy_d[4:6]), int(buy_d[6:]))
    sell = to_integer(buy_d+timedelta(days=hold_d))
    base = [20160101, 20160630, 20170102, 20170630, 20180102, 20180630, 
            20190102, 20190630, 20200102, 20200630, 20210101]
    for i in range(len(base)) :
        if sell < base[i] :
            if buy > base[i-1] :
                return hold_d*0.6
            else :
                return working_day(buy, base[i-1])

In [25]:
train_df['hist_d'] = train_df.apply(lambda x : hist_d_half(x['byn_dt'], x['hold_d']), axis = 1)

### Drop Features

In [26]:
train_df = train_df.drop(['act_id', 'iem_cd', 'code', 'byn_dt', 'mrz_pdt_tp_sgm_cd', 'stk_dit_cd', 'mkt_pr_tal_scl_tp_cd'], axis = 1)

In [27]:
used_features = train_df.columns.drop('hold_d')

In [28]:
test_df = test_df[used_features]

In [29]:
train_df.columns

Index(['hold_d', 'sex_dit_cd', 'cus_age_stn_cd', 'ivs_icn_cd',
       'cus_aet_stn_cd', 'lsg_sgm_cd', 'tco_cus_grd_cd', 'tot_ivs_te_sgm_cd',
       'mrz_btp_dit_cd', 'iem_krl_nm', 'btp_cfc_cd', 'past_d', 'hist_d',
       'volume', 'iem_cd_te', 'ivs_past_d_mean'],
      dtype='object')

In [30]:
test_df.columns

Index(['sex_dit_cd', 'cus_age_stn_cd', 'ivs_icn_cd', 'cus_aet_stn_cd',
       'lsg_sgm_cd', 'tco_cus_grd_cd', 'tot_ivs_te_sgm_cd', 'mrz_btp_dit_cd',
       'iem_krl_nm', 'btp_cfc_cd', 'past_d', 'hist_d', 'volume', 'iem_cd_te',
       'ivs_past_d_mean'],
      dtype='object')

## Scaling

In [31]:
ss = StandardScaler()

In [32]:
train_df.head()

,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,past_d,hist_d,volume,iem_cd_te,ivs_past_d_mean
0,11,1,9,3,2,9,5,5,8,101,1,195.0,6.6,75303.913610,23.934985,39.551724
1,80,1,9,3,2,9,5,5,8,1361,9,86.0,48.0,48284.637327,35.437256,55.771863
2,5,1,9,3,2,9,5,5,8,2530,12,6.0,3.0,414986.983700,20.464120,25.934524
3,22,1,9,3,2,9,5,5,8,1969,8,22.0,13.2,883043.104319,19.122137,24.164948
4,324,1,9,3,2,9,5,5,8,1696,10,347.0,213.0,114843.096985,27.660819,42.586207


In [33]:
scale_features = ['volume']

In [34]:
ss.fit(train_df[scale_features])

StandardScaler()

In [35]:
train_df[scale_features] = ss.transform(train_df[scale_features])

In [36]:
test_df[scale_features] = ss.transform(test_df[scale_features])

## 05. XGBoost
model : 기본 XGBoost 모델  
xgb_grid : GridSearch libarary를 통해서 model 최적 파라미터 구한 모델  
* 둘 중 하나로 테스트해보면 되는데 xgb_grid는 정확도가 올라가는 대신 시간 매우 오래 걸림

In [38]:
col = train_df.columns

In [39]:
col = col.drop('hold_d')

In [40]:
TARGET = 'hold_d'

In [41]:
X_train, X_valid, y_train, y_valid = train_test_split(train_df[col], train_df[TARGET], test_size = 0.2)

In [42]:
model = xgb.XGBRegressor(n_estimators=1000, max_depth=8, min_child_weighteta=0.1, subsample=0.7, colsample_bytree=0.8)

#### GridSearch

In [64]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'eta' :[0.05, 0.1],
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7, 8],
              'min_child_weight': [1, 2, 4, 6],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500, 1000]}

xgb_grid = GridSearchCV(model,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

In [ ]:
model.fit(X_train, y_train, verbose=False)

[22:05:15] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [65]:
xgb_grid.fit(X_train, y_train, verbose=False)

Fitting 2 folds for each of 192 candidates, totalling 384 fits
[13:56:57] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:57] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:57:29] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:29] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.

[13:56:57] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:57] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:58:02] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:58:02] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:58:35] WARNING: ../src/objective/regression_obj.cu:171: 

[13:56:57] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:57] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:57:29] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:29] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:58:34] WARNING: ../src/objective/regression_obj.cu:171: 

[13:56:57] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:57] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:58:04] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:58:04] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:59:09] WARNING: ../src/objective/regression_obj.cu:171: 

[13:56:57] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:56:57] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:57:30] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[13:57:30] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:58:37] WARNING: ../src/objective/regression_obj.cu:171: 

...


[15:00:30] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:00:30] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:02:08] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:02:08] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:02:55] WARNING: ../src/objective/regression_obj.cu:171:

[15:22:38] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:22:38] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




GridSearchCV(cv=2,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=0.8, gamma=0, gpu_id=-1,
                                    importance_type='gain',
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=8, min_child_weight=1,
                                    min_child_weighteta=0.1, missing=nan,
                                    monotone_constraints='()',
                                    n_estimators=...
                                    scale_pos_weight=1, subsample=0.7,
                                    tree_method='exact', validate_parameters=1,
                                    verbosity=None),
             n_jobs=5,
             param_grid={'colsample_bytree': [0.7], 'eta': [0.05,


[15:02:56] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:02:56] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:03:53] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:03:53] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:05:45] WARNING: ../src/objective/regression_obj.cu:171:


[15:04:10] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:10] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:05:06] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:05:06] WARNING: ../src/learner.cc:573: 
Parameters: { "min_child_weighteta", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:07:00] WARNING: ../src/objective/regression_obj.cu:171:

In [42]:
import joblib

In [67]:
joblib.dump(xgb_grid, 'xgb_gridcv.pkl')

['xgb_gridcv.pkl']

In [ ]:
xgb.plot_importance(model)

In [85]:
xgb_grid.score(X_train, y_train)   

NameError: name 'xgb_grid' is not defined

In [46]:
model.score(X_train, y_train)

0.9964943765502177

## Predict Score

In [47]:
pred = model.predict(X_valid)

In [48]:
pred = xgb_grid.predict(X_valid)

NameError: name 'xgb_grid' is not defined

In [48]:
y_valid.values

array([ 36,  11, 351, ...,   1,   4,   1])

In [49]:
np.round(pred)

array([ 34.,  11., 320., ...,   1.,   4.,   2.], dtype=float32)

In [50]:
np.sqrt(mean_squared_error(y_valid.values, np.round(pred)))

10.935636059620293

## 04. Submission

In [51]:
submission = pd.read_csv(f"{path}"+"sample_submission.csv")

In [52]:
y_pred = xgb_grid.predict(test_df) 

NameError: name 'xgb_grid' is not defined

In [53]:
y_pred = model.predict(test_df)

In [54]:
y_pred

array([324.46936 , 559.86914 , 245.52022 , ..., 993.3376  ,  22.953638,
        12.035654], dtype=float32)

In [55]:
result = []
for i in y_pred:
    result.append(i)

In [56]:
result

[324.46936,
 559.86914,
 245.52022,
 388.55136,
 13.035536,
 239.70468,
 197.5787,
 953.48737,
 160.35043,
 690.60626,
 289.50058,
 1130.4061,
 634.1023,
 10.447605,
 19.574152,
 3.293285,
 266.00723,
 160.99341,
 488.36142,
 24.674355,
 18.423527,
 1160.0269,
 627.1834,
 42.061314,
 5.2818637,
 98.990105,
 97.797356,
 464.87674,
 667.93225,
 483.07812,
 120.01333,
 102.33276,
 1023.2189,
 173.92992,
 22.964636,
 159.42361,
 189.92212,
 1148.1582,
 219.42473,
 1122.5618,
 1144.7942,
 602.28467,
 1117.166,
 1087.9565,
 1102.295,
 883.8406,
 1097.1288,
 1091.1289,
 1125.3423,
 1156.9844,
 1135.4799,
 1141.8762,
 236.76968,
 888.312,
 916.72095,
 300.07468,
 246.51154,
 878.01666,
 870.7318,
 951.44995,
 141.04333,
 914.53265,
 896.64124,
 941.2244,
 850.4501,
 893.5004,
 918.45807,
 179.51913,
 225.97302,
 902.738,
 952.68365,
 904.3088,
 953.29974,
 1096.109,
 23.198318,
 264.21844,
 56.53227,
 40.76687,
 245.63571,
 18.00724,
 2.1677666,
 1082.6902,
 1093.8314,
 1027.7638,
 18.851797,


In [57]:
submission["hold_d"] = np.round(result)

In [58]:
submission

,submit_id,hold_d
0,IDX00001,324.0
1,IDX00002,560.0
2,IDX00003,246.0
3,IDX00004,389.0
4,IDX00005,13.0
...,...,...
70591,IDX70592,47.0
70592,IDX70593,208.0
70593,IDX70594,993.0
70594,IDX70595,23.0


In [59]:
submission.to_csv("./result/xgboost(iem_cd_target_encoding lib with hist_d volume past_d(mean) ivs past_d hist_d(6) stdscaling(volume, hist_d, past_d)).csv", index = False)

In [64]:
''' 71점 columns
Index(['hold_d', 'sex_dit_cd', 'cus_age_stn_cd', 'ivs_icn_cd',
       'cus_aet_stn_cd', 'mrz_pdt_tp_sgm_cd', 'lsg_sgm_cd', 'tco_cus_grd_cd',
       'tot_ivs_te_sgm_cd', 'mrz_btp_dit_cd', 'iem_krl_nm', 'btp_cfc_cd',
       'mkt_pr_tal_scl_tp_cd', 'stk_dit_cd', 'past_d', 'hist_d', 'volume',
       'iem_cd_te', 'ivs_past_d_mean'],
      dtype='object')

'''

" 71점 columns\nIndex(['hold_d', 'sex_dit_cd', 'cus_age_stn_cd', 'ivs_icn_cd',\n       'cus_aet_stn_cd', 'mrz_pdt_tp_sgm_cd', 'lsg_sgm_cd', 'tco_cus_grd_cd',\n       'tot_ivs_te_sgm_cd', 'mrz_btp_dit_cd', 'iem_krl_nm', 'btp_cfc_cd',\n       'mkt_pr_tal_scl_tp_cd', 'stk_dit_cd', 'past_d', 'hist_d', 'volume',\n       'iem_cd_te', 'ivs_past_d_mean'],\n      dtype='object')\n\n"